# Introduction

This is a Jupyter notebook. It allows us to run code (in this case Python) alongside text. You can run code cell by selecting the cell and then clicking the run button or holding `Shift+Enter`. Output from the cell will be displayed below it.

The cell below imports external Python code for use here (specifically a package `trimesh` for loading and displaying 3D meshes in the notebook and some utilities for viewing file contents).

In [ ]:
import trimesh
from utils import *
from yggdrasil.runner import run

The cell below uses the `trimesh` package to load an example 3D mesh (plants-2) that will be used in today's exercises. After running this cell, a display of the 3D mesh should appear that you can rotate by clicking and dragging (note that you have to click a second time to stop manipulating the mesh).

In [ ]:
fname = 'meshes/plants-2.obj'
mesh = trimesh.load_mesh(fname)
mesh.show()

# Toy Plant Model

For the exercises today, we will be using a toy functional-structural plant model that makes the above plant mesh grow over time.

In the first version of this model, plant_v0.py (shown below), the growth is very simple and scales directly with time.

In [ ]:
from models import plant_v0
print_python_source(plant_v0.run)

The cell below runs this model directly (it is written in Python) and displays the resulting mesh at the end of the run. As you can see the mesh has 'grown' vertically.

In [ ]:
mesh = plant_v0.run(mesh, 0, 28, 1)
mesh.show()

However, we can also run the model using `yggdrasil` if we have a yaml configuration file for it. The cell below displays the configuration file for this model.

In [ ]:
print_yaml('yamls/plant_v0.yml')

The following code allows yggdrasil to run the model using the Python API. This is usually done via the `yggrun` command line utility (e.g. `yggrun yamls/plant_v0.yml`). Because this method of running the model requires loading the `yggdrasil` library, it takes a bit longer to run, but would not be significantly larger for a model with a more realistic (and computationally intensive) calculation.

In [ ]:
run('yamls/plant_v0.yml')

But now we want to add information about processes at other scales to our plant model...

# Call to Light Model

We can add information by calling other models as functions. For example, consider the toy light model below. It is a function that takes height and time as input and returns the light. 

Height can be a scalar or an array, but it is computationally more efficient to pass the 

In [ ]:
from models import light
print_python_source(light.light)

Because the light model is a function, yggdrasil can automatically wrap it with interface send and receive calls. Therefore, the YAML specification file for this model is relativly simple:

In [ ]:
print_yaml('yamls/light.yml')

Then we can add a interface call to the plant model "client" that calls the light model "server". In addition to the client comm, I have also added an output comm for the light values so that we can plot the light values.

In [ ]:
from models import plant_v1
print_python_source_diff(plant_v0.run, plant_v1.run)

The YAML specification file for this version of the plant model differs in that it specifies that it is a client of the model with the name "light" and that the light output should be directed to a file if it is not connected to a model.

In [ ]:
print_yaml_diff('yamls/plant_v0.yml', 'yamls/plant_v1.yml')

We can run the integration of these two models by providing the paths to the YAML specification models to the yggdrasil API. The command to run this integration on the command line would be `yggrun yamls/plant_v1.yml yamls/light.yml`.

In [ ]:
run(['yamls/plant_v1.yml', 'yamls/light.yml'], production_run=True)

The cell below displays the resulting mesh with color mapped to the light. We can see that the growth tracked by this version of the model is much more subtle.

In [ ]:
display_last_timestep(with_light=True)

## The light model is easily replaced with other models

Feel free to try running one or more of them. They have been coded to produce the same results as the Python version of the light model.

### C++ Model

In [ ]:
print_yaml('yamls/light_cpp.yml')
print_source('models/light.cpp')
run(['yamls/plant_v1.yml', 'yamls/light_cpp.yml'], production_run=True)
display_last_timestep(with_light=True)

### R Model

In [ ]:
print_yaml('yamls/light_R.yml')
print_source('models/light.R')
run(['yamls/plant_v1.yml', 'yamls/light_R.yml'], production_run=True)
display_last_timestep(with_light=True)

# Time Step Synchronization

In addition to allowing models to call other models as functions, yggdrasil also provides a method of synchronizing data across time dependent models, even if the models have different time steps. To see how this works in practice, the following shows how we could connect the plant model from above with a time-dependent root growth model.

In the version of the plant model displayed by the cell below, there is now a `YggTimesync` comm that can be called (using the same syntax as the `YggRpcClient` to get data from other time dependent models (in this case a root growth model).

In [ ]:
from models import plant_v2
print_python_source_diff(plant_v1.run, plant_v2.run)

In [ ]:
print_yaml_diff('yamls/plant_v1.yml', 'yamls/plant_v2.yml')

In [ ]:
from models import roots
print_python_source(roots.run)

In [ ]:
print_yaml('yamls/roots.yml')

In [ ]:
run(['yamls/plant_v2.yml', 'yamls/light.yml', 'yamls/roots.yml'], production_run=True)

In [ ]:
display_last_timestep(with_light=True)
plot_mass()

# Some Notes
You may have noticed a few keyword options passed to the `run` API function:

`production_run`
: When set to `True`, yggdrasil turns of several safe guards that increase run-time. These include things like checking data formats and validating inputs/outputs to/from framework components. It is highly recommended, that `production_run` is only set to `True` when you are done testing an integration and are ready for a "production run" that requires higher performance.